In [68]:
import numpy as np
import pandas as pd
import time

In [69]:
import csv
with open('Q Learning data.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        print(row)

['music', 'times']
['1', '20']
['2', '5']
['3', '10']
['4', '17']
['5', '13']
['6', '15']
['7', '10']
['8', '30']
['9', '0']
['10', '30']
['11', '22']
['12', '25']
['13', '30']
['14', '7']
['15', '30']
['16', '1']
['17', '2']
['18', '5']
['19', '14']
['20', '0']


In [70]:
df = pd.read_csv('Q Learning data.csv')

In [71]:
len(df)

20

In [72]:
#music = ["M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12", "M13", "M14", "M15", "M16", "M17", "M18", "M19", "M20"]
music = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17 ,18, 19, 20]
times = [20, 5, 10, 17, 13, 15, 10, 30, 0, 30, 22, 25, 30, 7, 30, 1, 2, 5, 14, 0]

music_dict = {"music": music, "times": times}

music_df = pd.DataFrame(music_dict)

In [73]:
music_df

,music,times
0,1,20
1,2,5
2,3,10
3,4,17
4,5,13
5,6,15
6,7,10
7,8,30
8,9,0
9,10,30


In [74]:
N_STATES = 20   # 1维世界的寬度
ACTIONS = ['dislike', 'click', 'listen', 'like']     # 探索者的可用動作
EPSILON = 0.9   # 貪婪度 greedy
ALPHA = 0.1     # 學習率
GAMMA = 0.9    # 獎勵遞減值
MAX_EPISODES = 13   # 最大回合數
FRESH_TIME = 0.3    # 移動尖閣時間

In [75]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table初始全0
        columns=actions,    # columns對應的是行為名稱
    )
    return table

In [76]:
build_q_table(N_STATES, ACTIONS)

,dislike,click,listen,like
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [77]:
A_List = []
for i in range (len(music_df)):
    times = music_df["times"][i]
    if times == 0:
        A = 'dislike'
        A_List.append(A)
    elif 0 < times < 10:
        A = 'click'
        A_List.append(A)
    elif 10 <= times < 30:
        A = 'listen'
        A_List.append(A)
    elif times >= 30:
        A = 'like'
        A_List.append(A)
print(A_List)

['listen', 'click', 'listen', 'listen', 'listen', 'listen', 'listen', 'like', 'dislike', 'like', 'listen', 'listen', 'like', 'click', 'like', 'click', 'click', 'click', 'listen', 'dislike']


In [135]:
# 在某個state地點，選擇行為
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :]  # 選出這個state的所有action值
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0):  # 非貪婪 or 或者這個 state 還沒有探索過
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax()    # 貪婪模式
    return action_name

In [171]:
def action(state, q_table):
    for i in range (len(music_df)):
        times = music_df["times"][i]
        if times == 0:
            A = 'dislike'
            A_List.append(A)
        elif 0 < times < 10:
            A = 'click'
            A_List.append(A)
        elif 10 <= times < 30:
            A = 'listen'
            A_List.append(A)
        elif times >= 30:
            A = 'like'
            A_List.append(A)
    return A

In [189]:
def get_env_feedback(S, A):
    
    for i in range (len(music_df)):
        times = music_df["times"][i]
        if times == 0:
            A = 'dislike'
            A_List.append(A)
        elif 0 < times < 10:
            A = 'click'
            A_List.append(A)
        elif 10 <= times < 30:
            A = 'listen'
            A_List.append(A)
        elif times >= 30:
            A = 'like'
            A_List.append(A)
        
    if A == 'dislike':     # dislike扣兩分
        if S == N_STATES - 1:   # terminate
            S_ = 'terminal' 
            R = -2
        else:
            S_ = S + 1
            R = -2
    elif A == 'click':     # click扣一分
        R = -1
        if S == 0:
            S_ = S 
        else:
            S_ = S - 1
    elif A == 'listen':   # listen加一分
        R = 1
        if S == 0:
            S_ = S 
        else:
            S_ = S - 1
    elif A == 'like':    # like加兩分
        R = 2
        if S == 0:
            S_ = S
        else:
            S_ = S - 1
    return S_, R

In [190]:
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print(interaction)
        print('=====================================================')
        #print('\r{}'.format(interaction), end='')
        #time.sleep(2)
        #print('\r                                ', end='')
    
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print(interaction)
        #print('\r{}'.format(interaction), end='')
        #time.sleep(FRESH_TIME)

In [191]:
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)  # 初始 q table
    
    for episode in range(MAX_EPISODES):     # 回合數
        step_counter = 0
        S = music_df["music"][0]-1   # 回合初始位置
        #print(S) S從0開始
        
        is_terminated = False   # 是否回合結束
        update_env(S, episode, step_counter)    # 環境更新
        
        while not is_terminated:

            A = choose_action(S, q_table)   # 選行為
            print("選行為：", A)
            S_, R = get_env_feedback(S, A)  # 實施行為並得到環境的反饋
            print("S_: ",S_)
            print("R: ",R)
            print()
            q_predict = q_table.loc[S, A]    # 估算的(狀態-行為)值
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # 實際的(狀態-行為)值 (回合沒結束)
            else:
                q_target = R     #  實際的(狀態-行為)值 (回合結束)
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  #  q_table更新
            S = S_  # 移動到下一個 state
            #print(q_table)
            
            update_env(S, episode, step_counter+1)  # 環境更新

            step_counter += 1
    
    return q_table

In [192]:
if __name__ == "__main__":
    q_table = rl()
    #print(type(A))
    print('=====DONE=====')

o------------------T
選行為： listen
S_:  1
R:  -2

-o-----------------T
選行為： listen
S_:  2
R:  -2

--o----------------T
選行為： dislike
S_:  3
R:  -2

---o---------------T
選行為： dislike
S_:  4
R:  -2

----o--------------T
選行為： dislike
S_:  5
R:  -2

-----o-------------T
選行為： dislike
S_:  6
R:  -2

------o------------T
選行為： listen
S_:  7
R:  -2

-------o-----------T
選行為： listen
S_:  8
R:  -2

--------o----------T
選行為： dislike
S_:  9
R:  -2

---------o---------T
選行為： click
S_:  10
R:  -2

----------o--------T
選行為： click
S_:  11
R:  -2

-----------o-------T
選行為： dislike
S_:  12
R:  -2

------------o------T
選行為： like
S_:  13
R:  -2

-------------o-----T
選行為： like
S_:  14
R:  -2

--------------o----T
選行為： click
S_:  15
R:  -2

---------------o---T
選行為： dislike
S_:  16
R:  -2

----------------o--T
選行為： listen
S_:  17
R:  -2

-----------------o-T
選行為： dislike
S_:  18
R:  -2

------------------oT
選行為： listen
S_:  19
R:  -2

-------------------o
選行為： click
S_:  terminal
R:  -2

Episode 1: total_steps 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  import sys


 click
S_:  5
R:  -2

-----o-------------T
選行為： listen
S_:  6
R:  -2

------o------------T
選行為： click
S_:  7
R:  -2

-------o-----------T
選行為： listen
S_:  8
R:  -2

--------o----------T
選行為： like
S_:  9
R:  -2

---------o---------T
選行為： dislike
S_:  10
R:  -2

----------o--------T
選行為： click
S_:  11
R:  -2

-----------o-------T
選行為： click
S_:  12
R:  -2

------------o------T
選行為： click
S_:  13
R:  -2

-------------o-----T
選行為： dislike
S_:  14
R:  -2

--------------o----T
選行為： dislike
S_:  15
R:  -2

---------------o---T
選行為： click
S_:  16
R:  -2

----------------o--T
選行為： listen
S_:  17
R:  -2

-----------------o-T
選行為： like
S_:  18
R:  -2

------------------oT
選行為： click
S_:  19
R:  -2

-------------------o
選行為： listen
S_:  terminal
R:  -2

Episode 7: total_steps = 20
o------------------T
選行為： click
S_:  1
R:  -2

-o-----------------T
選行為： like
S_:  2
R:  -2

--o----------------T
選行為： dislike
S_:  3
R:  -2

---o---------------T
選行為： click
S_:  4
R:  -2

----o--------------T
選行為： like


In [193]:
print('\r\nQ-table:')
q_table


Q-table:


,dislike,click,listen,like
0,-0.634682,-0.560000,-0.592400,-0.736580
1,-0.726682,-0.560000,-0.561620,-0.577966
2,-0.687800,-0.596936,-0.542000,-0.627554
3,-0.687800,-0.637906,-0.608600,-0.592400
4,-0.744194,-0.576200,-0.577820,-0.594020
5,-0.576200,-0.592400,-0.636140,-0.722000
6,-0.542000,-0.542000,-0.687800,-0.542000
7,0.000000,-0.757316,-0.972938,-0.577820
8,-0.592400,-0.687800,-0.636140,-0.576200
9,-0.560000,-0.722000,-0.576200,-0.576200
